# <p style="background-color: #ADEFD1FF; font-family: Pacifico, cursive; color: black; font-size: 150%; text-align: center; border-radius: 20px; padding: 10px 30px; display: block; width: 95%; margin: 0;">Baseline ---> Poisonous Mushrooms🤖</p>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import *
from sklearn.preprocessing import *

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import *
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None)

from sklearn.model_selection import StratifiedKFold
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import matthews_corrcoef
from collections import Counter
import joblib


n_splits = 10
SEED = 42

# <p style="background-color: #ADEFD1FF; font-family: Pacifico, cursive; color: black; font-size: 100%; text-align: left; border-radius: 20px; padding: 10px 30px; display: inline-block;">📂 Load Data</p>

In [2]:
%%time

#% sample_sub = pd.read_csv('/kaggle/input/playground-series-s4e8/sample_submission.csv')

train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')

# Original = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

train.drop(columns=['id'], inplace=True)

test.drop(columns=['id'], inplace=True)

#train = pd.concat([train,Original])

train = train [['cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season','class']]

CPU times: total: 8.78 s
Wall time: 9.75 s


In [3]:
train.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,class
0,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a,e
1,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w,p
2,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w,e
3,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u,e
4,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a,e


In [4]:
train.isnull().sum()

cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
class                         0
dtype: int64

In [5]:
test.isnull().sum()

cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

# <p style="background-color: #ADEFD1FF; font-family: Pacifico, cursive; color: black; font-size: 100%; text-align: left; border-radius: 20px; padding: 10px 30px; display: inline-block;"> Null Value Handling and Preprocess Data 📊 🔍</p>

In [6]:
print('Dup Values in Train')
print(train.duplicated().sum())
print('------------------------')

print('Dup Values in Test')
print(test.duplicated().sum())
print('------------------------')

Dup Values in Train
0
------------------------
Dup Values in Test
0
------------------------


In [7]:
# %%time

'''For Numeric Null Values I prefer the iterative imputer and For The Cat_null Values I Prefer the
Constant Imputation. Although The Tree Models and Gradient Models Can Handle The Null Values, But I Decied to Impute and
Move Further.
I Prefer The Ordinal Encoding For Data and Label Encoding For Target'''

def handle_missing_values(train, test, seed=None):
    
    num_c = [col for col in train.columns if train[col].dtype in ['float64', 'int64']]
    cat_c = [col for col in train.columns if train[col].dtype == 'object']
    if 'class' in cat_c:
        cat_c.remove('class')
    
    I = IterativeImputer(random_state=seed)
    train[num_c] = I.fit_transform(train[num_c])
    test[num_c] = I.transform(test[num_c])
    
    for col in cat_c:
        train[col].fillna('No_Record', inplace=True)
        test[col].fillna('No_Record', inplace=True)
    
    return train, test, cat_c

train, test, cat_c = handle_missing_values(train, test, seed=SEED)

# %%time

def align_columns(train, test):

    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    return train, test

target = train['class']
train_features = train.drop(columns=['class'], errors='ignore')

train_features_aligned, test_features_aligned = align_columns(train_features, test)

encoder = ce.OrdinalEncoder(cols=cat_c, handle_unknown='ignore')
train = encoder.fit_transform(train_features_aligned)
test = encoder.transform(test_features_aligned)

train['class'] = target

le = LabelEncoder()

train['class'] = le.fit_transform(train['class'])


# <p style="background-color: #ADEFD1FF; font-family: Pacifico, cursive; color: black; font-size: 100%; text-align: left; border-radius: 20px; padding: 10px 30px; display: inline-block;"> Basline Model 🔍</p>

In [8]:
%%time

X = train.drop('class',axis=1)
y = train['class']

def Train_ML(Model, X, y, test_data):
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    train_scores = []
    val_scores = []
    test_predictions = [] 
    models = []

    for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        
        model = Model
        
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        train_mcc = matthews_corrcoef(y_train, y_train_pred)
        train_scores.append(train_mcc)

        y_val_pred = model.predict(X_val)
        val_mcc = matthews_corrcoef(y_val, y_val_pred)
        val_scores.append(val_mcc)
        
        y_test_pred_proba = model.predict(test_data)
        test_predictions.append(y_test_pred_proba)
        
        models.append(model)

        print(f"Fold {fold}: Train MCC = {train_mcc:.6f}, Validation MCC = {val_mcc:.6f}")

    mean_train_mcc = np.mean(train_scores)
    mean_val_mcc = np.mean(val_scores)

    print(f"\nMean Train MCC: {mean_train_mcc:.6f}")
    print(f"Mean Validation MCC: {mean_val_mcc:.6f}")

    return model,test_predictions

CPU times: total: 172 ms
Wall time: 222 ms


# <p style="background-color: #ADEFD1FF; font-family: Pacifico, cursive; color: black; font-size: 100%; text-align: left; border-radius: 20px; padding: 10px 30px; display: inline-block;"> XGB Classifier 🚀</p>

In [9]:
%%time

# """Public Lb : 0.98489"""

Xparams_s1 = {'n_estimators': 1396, 'max_depth': 19, 'learning_rate': 0.010455050159676566, 'subsample': 0.8006842727555243, 
  'colsample_bytree': 0.5001438770455072, 'colsample_bylevel': 0.8027576507794217, 'min_child_weight': 5,
  'reg_alpha': 1.1586967014672253e-08, 'reg_lambda': 3.3517458803447213e-06, 'gamma': 0.01841032988451454}


xgb_s1 = XGBClassifier(**Xparams_s1, random_state=SEED,tree_method='gpu_hist')   
xgb_model_s1, xgb_preds_s1 = Train_ML(xgb_s1, X, y ,test)

joblib.dump(xgb_model_s1, 'xgb_model_s2.pkl')

Fold 1: Train MCC = 0.987311, Validation MCC = 0.984390
Fold 2: Train MCC = 0.987234, Validation MCC = 0.985166
Fold 3: Train MCC = 0.987266, Validation MCC = 0.984573
Fold 4: Train MCC = 0.987261, Validation MCC = 0.985179
Fold 5: Train MCC = 0.987267, Validation MCC = 0.984856
Fold 6: Train MCC = 0.987265, Validation MCC = 0.984947
Fold 7: Train MCC = 0.987244, Validation MCC = 0.984926
Fold 8: Train MCC = 0.987330, Validation MCC = 0.984510
Fold 9: Train MCC = 0.987264, Validation MCC = 0.984882
Fold 10: Train MCC = 0.987260, Validation MCC = 0.984965

Mean Train MCC: 0.987270
Mean Validation MCC: 0.984840
CPU times: total: 1h 19min 21s
Wall time: 1h 39min 14s


['xgb_model_s2.pkl']

# <p style="background-color: #ADEFD1FF; font-family: Pacifico, cursive; color: black; font-size: 100%; text-align: left; border-radius: 20px; padding: 10px 30px; display: inline-block;"> Submission 🚀</p>

In [12]:
test_df = pd.read_csv('test.csv')

xgb_test = np.mean(xgb_preds_s1, axis=0)

xgbpreds = (xgb_test >= 0.5).astype(int)
x_f = le.inverse_transform(xgbpreds)

sub_F = pd.DataFrame({
    'id' : test_df['id'],
    'class': x_f
})

sub_F.to_csv('Submission.csv', index=False)
print(sub_F.head())

        id class
0  3116945     e
1  3116946     p
2  3116947     p
3  3116948     p
4  3116949     e
